In [66]:
import numpy as np

inputs = [
    [1, 2, 3, 2.5], 
    [2., 5., -1., 2], 
    [-1.5, 2.7, 3.3, -0.8]]

weights = [
    [0.2, 0.8, -0.5, 1],
    [0.5, -0.91, 0.26, -0.5],
    [-0.26, -0.27, 0.17, 0.87]]

biases = [2, 3, 0.5]

weights2 = [
    [0.1, -0.14, 0.5],
    [-0.5, 0.12, -0.33],
    [-0.44, 0.73, -0.13]]

biases2 = [-1, 2, -0.5]

layer1_outputs = np.dot(inputs, np.array(weights).T) + biases
layer2_outputs = np.dot(layer1_outputs, np.array(weights2).T) + biases2

print(layer2_outputs)

[[ 0.5031  -1.04185 -2.03875]
 [ 0.2434  -2.7332  -5.7633 ]
 [-0.99314  1.41254 -0.35655]]


In [63]:
import numpy as np
from math import pi

def spiral_data(samples, classes):
    theta = np.sqrt(np.random.rand(samples)) * 2 * pi  # Radial distance scaled by random sqrt
    
    res = []
    for i in range(classes):
        r_a = 2 * theta + pi
        x1 = np.random.random(samples) + np.cos(theta) * r_a  # X-coordinates
        x2 = np.random.random(samples) + np.sin(theta) * r_a  # Y-coordinates
        labels = np.full(samples, i)  # Label for this class (i)
        
        # Stack x1, x2, and labels vertically and transpose to create the data for this class
        data = np.vstack((x1, x2, labels)).T
        
        # Append the data to the result list
        res.append(data)
        
    # Concatenate all class data into a single array
    res = np.vstack(res)
    
    # Shuffle the dataset
    np.random.shuffle(res)

    # Split the dataset into X (features) and y (labels)
    X = res[:, :2]
    y = res[:, 2].astype(int)  # Convert labels to int
    
    return X, y


In [52]:
class Layer_Dense:
    
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))
    
    def forward(self, inputs):
        self.output = np.dot(inputs, self.weights) + self.biases

In [70]:
class Activation_ReLU:
    def forward(self, inputs):
        self.output = np.maximum(0, inputs)

In [76]:
class Activation_Softmax:
    def forward(self, inputs):
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        self.output = probabilities

In [69]:
X, y = spiral_data(samples=100, classes=3)
# Create Dense layer with 2 input features and 3 output values
dense1 = Layer_Dense(2, 3)
# Create ReLU activation (to be used with Dense layer):
activation1 = Activation_ReLU()
# Make a forward pass of our training data through this layer
dense1.forward(X)
# Forward pass through activation func.
# Takes in output from previous layer
activation1.forward(dense1.output)

print(activation1.output[:5])

[[0.         0.11808871 0.02220027]
 [0.09509251 0.         0.        ]
 [0.069852   0.         0.        ]
 [0.09277925 0.         0.        ]
 [0.         0.19859809 0.04831757]]


In [77]:
softmax = Activation_Softmax()
softmax.forward([
    [1, 2, 3],
    [4, 5, 6]
])
print(softmax.output)

[[0.09003057 0.24472847 0.66524096]
 [0.09003057 0.24472847 0.66524096]]
